In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
import os
os.chdir("../../../"), os.getcwd()

(None, '/Users/goldyrana/mess/deep_learning/projects/rag')

In [2]:
cred = credentials.Certificate("credentials/firebase_credentials.json")
firebase_admin.initialize_app(cred)
firestore_db = firestore.client()

In [ ]:
def store_user_chat(user_id, chat_id):
    user_ref = firestore_db.collection("users").document(user_id)
    user_ref.update({"chat_id": chat_id})

In [8]:
firestore_db.collection("user_chats").document("id_1").get()

In [12]:
get_user_chat_history("user_chats")

[]

In [5]:
def get_user_chat_history(user_id, limit=5):
    """Retrieves the last few chat messages of a user from Firestore"""

    chat_ref = firestore_db.collection("user_chats").document(user_id).collection("history")

    # Fetch the last `limit` messages sorted by timestamp (latest first)
    chat_docs = chat_ref.order_by("timestamp", direction=firestore.Query.DESCENDING).limit(limit).stream()

    history = []
    for doc in chat_docs:
        history.append(doc.to_dict())

    return history

In [6]:
get_user_chat_history("default_user_1")

[]

In [9]:
x = firestore_db.collection("user_chats").document("default_user_1").collection("history")


In [18]:
for i in x.list_documents():
    print(i.id)

wCHxgsTvT7Ifx7MYp8Gs


In [20]:
x.document("wCHxgsTvT7Ifx7MYp8Gs").get().to_dict()

{'query': "user's input question", 'response': 'llm response'}

In [ ]:
# Setup Firebase Firestore
PROJECT_ID = "langchain-a5989"
SESSION_ID = "user_session_new"  # This could be a username or a unique ID
COLLECTION_NAME = "chat_history"

# Initialize Firestore Client
print("Initializing Firestore Client...")
client = firestore.Client(project=PROJECT_ID)

# Initialize Firestore Chat Message History
print("Initializing Firestore Chat Message History...")
chat_history = FirestoreChatMessageHistory(
    session_id=SESSION_ID,
    collection=COLLECTION_NAME,
    client=client,
)
print("Chat History Initialized.")
print("Current Chat History:", chat_history.messages)

# Initialize Chat Model
model = ChatOpenAI()

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
import os
os.chdir("../../../"), os.getcwd()

(None, '/Users/goldyrana/mess/deep_learning/projects/rag')

In [2]:
cred = credentials.Certificate("credentials/firebase_credentials.json")
firebase_admin.initialize_app(cred)
client = firestore.client()

In [3]:
from langchain_google_firestore import FirestoreChatMessageHistory
chat_history = FirestoreChatMessageHistory(
    session_id="user_session_new",
    collection="chat_history",
    client=client,
)

In [6]:
chat_history._load_messages()

In [8]:
from datetime import datetime


In [9]:
timestamp = datetime.utcnow().isoformat()

In [14]:
chat_history.add_user_message("hello my name is rana")
chat_history.add_ai_message("hello goldy, how can I help you today?")

In [16]:
chat_history.messages[-5:]

[AIMessage(content='hello goldy, how can I help you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello my name is goldy', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hello goldy, how can I help you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello my name is rana', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hello goldy, how can I help you today?', additional_kwargs={}, response_metadata={})]

In [ ]:
# Initialize Firestore Client
print("Initializing Firestore Client...")
client = firestore.Client(project=PROJECT_ID)


In [ ]:
from dataclasses import dataclass
from backend.src.constants import FIREBASE_CREDENTIALS_PATH # This can be hidden using google.cloud secretmanager 


@dataclass
class FireStoreConfig:
    firebase_credentials_path:str
    session_id: str

class ConfigurationManager:
    def __init__(self, config: FireStoreConfig, chat_history: str=None):
        self.config = config
        if chat_history is None:
            self.collection = "chat_history"
        self.chat_history = chat_history

    def get_firebase_params(self):
        cred = FireStoreConfig(
            firebase_credentials_path=self.config.firebase.firebase_credentials_path,
            session_id=self.config.firebase.session_id
        )
        return cred
    
class FireStore:
    def __init__(self, config: FireStoreConfig):
        self.config = config
      
    def get_firestore_client(self):
        cred = credentials.Certificate(self.config.firebase_credentials_path)
        firebase_admin.initialize_app(cred)
        return firestore.client()
    
    def get_chat_history(self, user_id, limit=5):
        chat_history=FirestoreChatMessageHistory(
            session_id=self.config.session_id,
            collection="chat_history",
            client=self.get_firestore_client()
        )
        return chat_history
        
    def load_messages(self, chat_history, limit=5):
        return chat_history.messages[-limit:]
    
    def add_user_message(self, human_message, ai_message) -> None:
        chat_history.add_user_message(human_message)
        chat_history.add_ai_message(ai_message)